In [1]:
from sqlalchemy import create_engine
import sqlalchemy
import pandas as pd
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv


In [2]:
from oceanoobsbrasil.db import GetData
%load_ext autoreload
%autoreload 2

In [4]:
bd = GetData()

In [4]:
bd.get(table='stations', institution=['=', 'simcosta'], data_type=['=', 'buoy'])

,id,name,lat,lon,data_type,institution,url
0,897,RJ3,-22.9831,-43.1745,buoy,simcosta,3
1,898,RS2,-32.1346,-52098.0000,buoy,simcosta,6
2,899,RS5,-32296.0000,-52.0239,buoy,simcosta,10
3,900,RJ4,-22.9715,-43.1522,buoy,simcosta,12
4,901,RS3,-32195.0000,-52.0794,buoy,simcosta,13
5,902,RS4,-32.2454,-52.0954,buoy,simcosta,14
6,903,BA1,-12.9895,-37.4582,buoy,simcosta,15


In [8]:
df = bd.get(table='data_stations', station_id=['in', [902]])

In [9]:
df.shape

(94, 22)

In [7]:
bd.post(table='data_stations', df=df)

DELETE FROM data_stations WHERE true AND station_id in (902) AND date_time >= '2021-09-06 00:21:00'


In [6]:
import time
import datetime
import operator
import urllib.request, json

import numpy as np
from oceanoobsbrasil.buoys.simcosta import Simcosta
from oceanoobsbrasil.tides.simcosta import SimcostaTide

In [7]:
s = Simcosta()

In [ ]:
s.get()

210
    atmp                 date_time
0   None 2022-03-04 20:01:30+00:00
1   None 2022-03-04 21:01:30+00:00
2   None 2022-03-04 22:01:30+00:00
3   None 2022-03-04 23:01:30+00:00
4   None 2022-03-05 00:01:30+00:00
5   None 2022-03-05 01:01:30+00:00
6   None 2022-03-05 02:01:30+00:00
7   None 2022-03-05 03:01:30+00:00
8   None 2022-03-05 04:01:30+00:00
9   None 2022-03-05 05:01:30+00:00
10  None 2022-03-05 06:01:30+00:00
11  None 2022-03-05 07:01:30+00:00
12  None 2022-03-05 08:01:30+00:00
13  None 2022-03-05 09:01:30+00:00
14  None 2022-03-05 10:01:30+00:00
15  None 2022-03-05 11:01:30+00:00
16  None 2022-03-05 12:01:30+00:00
17  None 2022-03-05 13:01:30+00:00
18  None 2022-03-05 14:01:30+00:00
19  None 2022-03-05 15:01:30+00:00
20  None 2022-03-05 16:01:30+00:00
21  None 2022-03-05 17:01:30+00:00
22  None 2022-03-05 18:01:30+00:00
ok
504
       pres  atmp    rh dewpt  wspd   wdir  gust  swvht  mxwvht    tp    sst  \
0   1014.53  None  None  None   NaN    NaN   NaN   0.71    1.27  12.5

In [37]:
s.stations.iloc[0].url

'3'

In [50]:
url_address = f"https://simcosta.furg.br/api/intrans_data?boiaID=510&type=json&time1={s.start_date}&time2={s.end_date}&params=Avg_Air_Press,Avg_Wnd_Dir_N,Gust_Sp,Avg_Dew,Avg_Sol_Rad,Avg_Air_Tmp,Hmt,Avg_Hmt,Avg_Wnd_Sp,Wv_H10,Wv_Havg,Wv_Hsig,Wv_HM0,Wv_Avg_Dir,Wv_Hmax,Wv_ZCN,Wv_Tp5,Wv_Tz,Wv_Tavg,Wv_T10,Wv_Tsig,Wv_Avg_Spread,Wv_Tp,Avg_Sal,Avg_W_Tmp1,Avg_W_Tmp2,Avg_Fluo,Avg_DO,Avg_DO,Avg_Turb"
with urllib.request.urlopen(url_address) as url:
    data = json.loads(url.read().decode())
    s.data = pd.DataFrame(data)

In [52]:
s.result = s.data

In [57]:
s.result['date_time'] = pd.to_datetime(s.result.timestamp)
s.result = s.result[[ 'Avg_Air_Press', 'Avg_Air_Tmp','Avg_Hmt', 'Avg_Dew',
       'Avg_Wnd_Sp', 'Avg_Wnd_Dir_N','Gust_Sp', 'Wv_Hsig', 'Wv_Hmax','Wv_Tp', 
       'Avg_W_Tmp1','Wv_Avg_Spread','Wv_Avg_Dir', 'date_time']]

In [55]:
s.result.columns = 
columns = ['pres', 'atmp', 'rh', 'dewpt', 'wspd', 'wdir', 'gust', 'swvht', 'mxwvht', 'tp', 'sst', 'wvspread', 'wvdir', 'date_time']

Index(['timestamp', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'SECOND',
       'Avg_Dew', 'Avg_Hmt', 'Hmt', 'Avg_Air_Tmp', 'Avg_Air_Press', 'Gust_Sp',
       'Avg_Wnd_Sp', 'Avg_Wnd_Dir_N', 'Wv_Hsig', 'Wv_Tp', 'Wv_Hmax', 'Wv_Havg',
       'Wv_Tavg', 'Wv_H10', 'Wv_T10', 'Wv_Avg_Dir', 'Wv_Avg_Spread', 'Wv_Tsig',
       'Wv_Tp5', 'Wv_HM0', 'Wv_ZCN', 'Avg_Sal', 'DO', 'Avg_W_Tmp1',
       'Avg_W_Tmp2', 'Turb', 'Fluo', 'date_time'],
      dtype='object')

In [40]:
columns = ['pres', 'atmp', 'rh', 'dewpt', 'wspd', 'wdir', 'gust', 'swvht', 'mxwvht', 'tp', 'sst', 'wvspread', 'wvdir', 'date_time']

,timestamp,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,Avg_Dew,Avg_Hmt,Hmt,Avg_Air_Tmp,Avg_Air_Press,Gust_Sp,Avg_Wnd_Sp,Avg_Wnd_Dir_N
0,2021-11-04T04:21:40.000Z,2021,11,4,4,21,40,16.0,79.1,None,19.9,NaN,NaN,NaN,NaN
1,2021-11-04T04:51:40.000Z,2021,11,4,4,51,40,17.6,86.5,None,19.9,1005.05,8.6,7.1,13.0
2,2021-11-04T05:21:40.000Z,2021,11,4,5,21,40,17.5,85.8,None,19.9,1004.84,8.2,6.7,5.0
3,2021-11-04T05:51:40.000Z,2021,11,4,5,51,40,17.4,85.3,None,20.0,1005.06,7.0,5.1,350.0
4,2021-11-04T06:21:40.000Z,2021,11,4,6,21,40,17.5,86.8,None,19.7,1005.08,3.8,2.9,352.0
5,2021-11-04T06:51:40.000Z,2021,11,4,6,51,40,17.6,88.0,None,19.6,1005.27,5.0,3.9,42.0
6,2021-11-04T07:21:40.000Z,2021,11,4,7,21,40,17.3,86.7,None,19.6,1005.39,4.1,3.4,38.0
7,2021-11-04T07:51:40.000Z,2021,11,4,7,51,40,17.8,89.7,None,19.6,1005.75,6.9,5.6,74.0
8,2021-11-04T08:21:40.000Z,2021,11,4,8,21,40,18.4,93.2,None,19.5,1005.80,7.4,6.1,91.0
9,2021-11-04T08:51:40.000Z,2021,11,4,8,51,40,18.3,93.7,None,19.3,1006.41,5.5,4.6,75.0


In [5]:
s = SimcostaTide()

NameError: name 'SimcostaTide' is not defined

In [5]:
s.get()

Nao ha dados para essa boia
Nao ha dados para essa boia
Nao ha dados para essa boia


KeyboardInterrupt: 

In [52]:
s = Simcosta()

In [53]:
s.get()

id                  897
name                RJ3
lat            -22.9831
lon            -43.1745
data_type          buoy
institution    simcosta
url                   3
Name: 0, dtype: object
http://simcosta.furg.br/api/metereo_data?boiaID=3&type=json&time1=1631069870&time2=1631156270&params=Average_wind_direction_N,Last_sampling_interval_gust_speed,Average_Dew_Point,Average_Pressure,Solar_Radiation_Average_Reading,Average_Air_Temperature,Instantaneous_Humidity,Average_Humidity,Average_wind_speed
http://simcosta.furg.br/api/oceanic_data?boiaID=3&type=json&time1=1631069870&time2=1631156270&params=H10,HAvg,Hsig_Significant_Wave_Height_m,HM0,Mean_Wave_Direction_deg,Hmax_Maximum_Wave_Height_m,ZCN,Tp5,TAvg,T10,Tsig,Mean_Spread_deg,TP_Peak_Period_seconds,Average_Salinity,Average_Temperature_deg_C,Average_Temperature_C,Average_CDOM_QSDE,Average_Chlorophyll_Fluorescence,Average_Dissolved_Oxygen,Average_Nephelometric_Turbidity_Unit_NTU,Cell_Average_Direction_N,Cell_Average_Magnitude_mm_s
Empty D

    YEAR MONTH DAY HOUR MINUTE SECOND Avg_Air_Press Avg_Air_Tmp    Hmt  \
0   2021     9   8    3     21     00       1018.53        None  45.20   
1   2021     9   8    3     51     00       1018.39        None  44.30   
2   2021     9   8    4     51     00       1017.51        None  44.00   
3   2021     9   8    5     21     00       1017.30        None  44.60   
4   2021     9   8    5     51     00       1016.99        None  43.80   
5   2021     9   8    6     51     00       1015.98        None  43.70   
6   2021     9   8    7     21     00       1015.71        None  44.60   
7   2021     9   8    7     51     00       1015.55        None  43.60   
8   2021     9   8    8     21     00       1015.67        None  44.50   
9   2021     9   8    9     21     00       1016.01        None  44.50   
10  2021     9   8    9     51     00       1016.16        None  43.70   
11  2021     9   8   10     21     00       1016.07        None  45.40   
12  2021     9   8   10     51     00 

ok2
id                  903
name                BA1
lat            -12.9895
lon            -37.4582
data_type          buoy
institution    simcosta
url                  15
Name: 6, dtype: object
http://simcosta.furg.br/api/metereo_data?boiaID=15&type=json&time1=1631069870&time2=1631156270&params=Average_wind_direction_N,Last_sampling_interval_gust_speed,Average_Dew_Point,Average_Pressure,Solar_Radiation_Average_Reading,Average_Air_Temperature,Instantaneous_Humidity,Average_Humidity,Average_wind_speed
http://simcosta.furg.br/api/oceanic_data?boiaID=15&type=json&time1=1631069870&time2=1631156270&params=H10,HAvg,Hsig_Significant_Wave_Height_m,HM0,Mean_Wave_Direction_deg,Hmax_Maximum_Wave_Height_m,ZCN,Tp5,TAvg,T10,Tsig,Mean_Spread_deg,TP_Peak_Period_seconds,Average_Salinity,Average_Temperature_deg_C,Average_Temperature_C,Average_CDOM_QSDE,Average_Chlorophyll_Fluorescence,Average_Dissolved_Oxygen,Average_Nephelometric_Turbidity_Unit_NTU,Cell_Average_Direction_N,Cell_Average_Magnitude_mm_s
 

In [40]:
                columns = ['YEAR','MONTH','DAY','HOUR','MINUTE','SECOND','Hmt',
                    'Avg_Wnd_Dir','M_Decl','Avg_W_Tmp1','Avg_Sal','Avg_Spre_N',
                    'Avg_Wv_Dir','Avg_Cel1_Mag','Avg_Cel1_Dir','Avg_Cel1_Dir_N',
                    'Avg_Turb','Avg_Chl','Avg_DO','ZCN','HM0','TAvg','Tp5',
                    'T10','HAvg','Tsig','CDOM','H10','Avg_Sol_Rad']



In [43]:
s.result.drop(columns=columns, inplace=True)

In [44]:
s.result.columns = ['pres', 'atmp', 'rh', 'dewpt', 'wspd', 'wdir', 'gust', 'swvht', 'mxwvht', 'tp', 'sst', 'wvspread', 'wvdir', 'date_time']

In [45]:
s.result = s.result.replace(to_replace =['None', 'NULL', ' ', ''],
                                        value =np.nan)

In [46]:
s.result['station_id'] = 898

In [ ]:
s.